In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Convert Labelme JSON (rectangle) to Ultralytics YOLO detection txt.
Assumptions:
- class "swd" -> id 0, "p" -> id 1 (modify CLASS_TO_ID if you add more classes)
- Only rectangles are used for detection (points are ignored)
"""

from pathlib import Path
import json, argparse

CLASS_TO_ID = {"insect": 0}   # add more classes if needed

def norm(val, denom): return float(val) / float(denom)

def rect_from_points(pts):
    xs = [p[0] for p in pts]; ys = [p[1] for p in pts]
    x1, x2 = min(xs), max(xs); y1, y2 = min(ys), max(ys)
    return x1, y1, x2, y2

def convert_one(json_path: Path, out_dir: Path):
    data = json.loads(Path(json_path).read_text())
    W, H = data["imageWidth"], data["imageHeight"]

    # collect rectangles only (ignore points for detection)
    rects = []
    for sh in data["shapes"]:
        t = sh.get("shape_type", "")
        lbl = sh.get("label", "")
        pts = sh.get("points", [])
        if t == "rectangle" and lbl in CLASS_TO_ID:
            rects.append({"label": lbl, "xyxy": rect_from_points(pts)})

    # if no rectangles, skip
    if not rects:
        return

    # write yolo txt (detection format)
    out_dir.mkdir(parents=True, exist_ok=True)
    stem = Path(data["imagePath"]).stem
    out_txt = out_dir / f"{stem}.txt"

    lines = []
    for r in rects:
        x1, y1, x2, y2 = r["xyxy"]
        cx = (x1 + x2) / 2.0; cy = (y1 + y2) / 2.0
        w = (x2 - x1); h = (y2 - y1)
        cxn, cyn, wn, hn = norm(cx, W), norm(cy, H), norm(w, W), norm(h, H)

        cls_id = CLASS_TO_ID[r["label"]]
        # Detection format: class_id center_x center_y width height
        line = f"{cls_id} {cxn:.6f} {cyn:.6f} {wn:.6f} {hn:.6f}"
        lines.append(line)

    out_txt.write_text("\n".join(lines))

if __name__ == "__main__":
    ann_dir = Path("/workspace/models/runs_yolov11_det_insect/cropped_objects")
    out_dir = Path("/workspace/models/runs_yolov11_det_insect/cropped_objects")
    
    # 确保输出目录存在
    out_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Converting Labelme JSON files from: {ann_dir}")
    print(f"Output directory: {out_dir}")
    
    converted_count = 0
    for jp in ann_dir.glob("*.json"):
        try:
            convert_one(jp, out_dir)
            converted_count += 1
            print(f"✓ Converted: {jp.name}")
        except Exception as e:
            print(f"[WARN] Failed to convert {jp.name}: {e}")
    
    print(f"\nConversion completed. Converted {converted_count} files.")

    # 验证转换结果
    txt_files = list(out_dir.glob("*.txt"))
    print(f"Generated {len(txt_files)} annotation files in {out_dir}")
    
    if txt_files:
        # 显示第一个文件的内容作为示例
        sample_file = txt_files[0]
        print(f"\nSample annotation content from {sample_file.name}:")
        print(sample_file.read_text())

Converting Labelme JSON files from: /workspace/models/runs_yolov11_det_insect/cropped_objects
Output directory: /workspace/models/runs_yolov11_det_insect/cropped_objects
✓ Converted: elderfarm22_2560_1536_3200_2176.json
✓ Converted: elderfarm22_2560_3072_3200_3712.json
✓ Converted: elderfarm22_3072_1536_3712_2176.json
✓ Converted: elderfarm22_3072_2560_3712_3200.json
✓ Converted: elderfarm22_3072_4096_3712_4736.json
✓ Converted: elderfarm22_3584_1536_4224_2176.json
✓ Converted: elderfarm22_3584_2560_4224_3200.json
✓ Converted: elderfarm11_1536_2048_2176_2688.json
✓ Converted: elderfarm11_1536_3584_2176_4224.json
✓ Converted: elderfarm11_1536_4096_2176_4736.json
✓ Converted: elderfarm11_2048_3072_2688_3712.json
✓ Converted: elderfarm11_2048_3584_2688_4224.json
✓ Converted: elderfarm11_2048_4096_2688_4736.json
✓ Converted: elderfarm11_2048_5120_2688_5760.json
✓ Converted: elderfarm11_2560_1536_3200_2176.json
✓ Converted: elderfarm11_2560_2048_3200_2688.json
✓ Converted: elderfarm11_2560_